In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
spark = SparkSession.builder.master("local[1]").appName("JAS").enableHiveSupport().getOrCreate()

In [1]:
from os import getcwd, listdir
from os.path import join, isfile, abspath, pardir
from time import time
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from shutil import copyfile
import re
from pickle import load, dump, HIGHEST_PROTOCOL
import sys
project_dir = join(abspath(join(getcwd(), pardir)))
sys.path.append(join(project_dir, "scripts"))
from run_queries import load_schema, load_data

In [2]:
scale = 1
schema_file = join(project_dir, "schema", "tpcds_schema.pickle")
data_dir = join(project_dir, "data_{}gb".format(scale))
queries_dir = join(project_dir, "queries_{}gb".format(scale))

In [4]:
schema_dict = load_schema(schema_file)
load_data(spark, schema_dict, data_dir)

21/10/31 00:12:56 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [5]:
def get_execute_query_time(file):
    with open(file, "rt") as f:
        try:
            spark.sql(f.read())
        except Exception as e:
            print("[[ error ]]: Failed to execute '{}' query due to:\n{}".format( file, e ))

In [6]:
queries = [x for x in listdir(queries_dir) if str(x).endswith(".sql") and isfile(join(queries_dir, x))]
digit_regex = re.compile(r"(?P<num>\d+)")

In [14]:
def modify_line(line):
    # as "<some text>" -> as some_text
    as_regex = re.compile(r"(as)\s*(?P<p1>\")\s*(?P<text>[^]]+)\s*(?P<p2>\")", re.S)
    line = as_regex.sub(lambda m: m.group().replace(m.group("text"), "{}".format("_".join(m.group("text").split(" "))), 1).replace(m.group("p1"), "", 1).replace(m.group("p2"), "", 1), line).replace("-","_to_")
    return line

In [15]:
s = '''                 (sr_returned_date_sk - ss_sold_date_sk <= 90) then 1 else 0 end)  as "61-90 days" '''
modify_line(s)

'                 (sr_returned_date_sk _to_ ss_sold_date_sk <= 90) then 1 else 0 end)  as 61_to_90_days '

In [7]:
for index, query in enumerate(queries):
    query_file = join(queries_dir, query)
    # query_no = int(digit_regex.findall(query)[0]) if len(digit_regex.findall(query)) else ""

    get_execute_query_time(query_file)

[[ error ]]: Failed to execute '/Users/mohammadzainabbas/Masters/ULB/Data-Warehouse/data-warehouse-project/queries_1gb/query_24.sql' query due to:

extraneous input 'with' expecting {<EOF>, ';'}(line 55, pos 0)

== SQL ==

-- start query 1 in stream 0 using template query24.tpl
with ssales as
(select c_last_name
,c_first_name
,s_store_name
,ca_state
,s_state
,i_color
,i_current_price
,i_manager_id
,i_units
,i_size
,sum(ss_sales_price) netpaid
from store_sales
,store_returns
,store
,item
,customer
,customer_address
where ss_ticket_number = sr_ticket_number
and ss_item_sk = sr_item_sk
and ss_customer_sk = c_customer_sk
and ss_item_sk = i_item_sk
and ss_store_sk = s_store_sk
and c_current_addr_sk = ca_address_sk
and c_birth_country <> upper(ca_country)
and s_zip = ca_zip
and s_market_id=7
group by c_last_name
,c_first_name
,s_store_name
,ca_state
,s_state
,i_color
,i_current_price
,i_manager_id
,i_units
,i_size)
select c_last_name
,c_first_name
,s_store_name
,sum(netpaid) paid
from ssales

In [5]:
def needs_to_split(query):
    # checks whether to split the query or not
    split_regex = re.compile(r"(;)\s*(with)", re.S)
    return len(split_regex.findall(query))

queries_dir = join(abspath(join(getcwd(), pardir)), "queries_1gb")
queries = [x for x in listdir(queries_dir) if x.endswith(".sql") and isfile(join(queries_dir, x))]

for query in queries:
    
    with open(join(queries_dir, query), 'rt') as f:
        if needs_to_split(f.read()):
            print("Query {} needs to be modified...".format(query))

Query query_24.sql needs to be modified...
Query query_23.sql needs to be modified...
Query query_39.sql needs to be modified...
Query query_14.sql needs to be modified...


##### Make Separate files

In [ ]:
with open(join(sql_queries_path, "tpcds_queries.sql"), "rt") as file:
    lines = file.readlines()
    query = ""
    query_no = 1
    for index, line in enumerate(lines):
        query_line = line.strip()
        query += "\n{}".format( query_line )
        
        # Separate each query
        if query_line.find("-- end query", 0, len(query_line)) != -1:
            
            with open(join(sql_queries_path, "query_{}.sql".format(query_no)), "w") as query_file:
                query_file.write(query)
                query = ""
                print("Query - {} written.".format(query_no))
                query_no = query_no + 1

##### Do the benchmarking

In [8]:
take_avg_run, skip_run = 5, 1
total_runs = take_avg_run + skip_run

In [9]:
def get_execute_query_time(file, debug=False):
    with open(file, "rt") as f:
        try:
            start_time = time()
            spark.sql(f.read())
            return time() - start_time      
        except Exception as e:
            print("[[ error ]]: Failed to execute '{}' query :(".format( file ))
            if debug:
                print("[[ error ]]:\n{}\n".format( e ))
            return None

In [ ]:
i = int(0)
for key, value in timings_dict.items():
    if len(value):
        i = i + 1
        print("="*20)
        print("\n{} - {}: {} secs\n".format(i, key, sum(value)/len(value)))
        # print("Avg.: {}".format( sum(value)/len(value) ))

###### Failed Queries

In [ ]:
failed_queries = [x for x in list(timings_dict.keys()) if not len(timings_dict[x])]
failed_queries

In [ ]:
modified_queries_path = join(project_dir, "modified_queries")

In [ ]:
total_queries = 4
for index, query in enumerate(failed_queries):
    if total_queries == index:
        break
    file_name = query.split('.')[0]
    copyfile(join(sql_queries_path, query), join(modified_queries_path, "{}_original.sql".format(file_name)))
    copyfile(join(sql_queries_path, query), join(modified_queries_path, "{}_modified.sql".format(file_name)))